# Deploying Merlin Query Tower with Vertex AI

* Create custom prediction routine (CPR)
* Upload query model to Vertex AI Model Registry
* Test registered models predictions

In [1]:
from google.cloud import aiplatform as vertex_ai
import os

PROJECT_ID = 'hybrid-vertex'             
LOCATION = 'us-central1' 
BUCKET = 'jt-merlin-scaling'
BUCKET_URI = 'gs://jt-merlin-scaling'

vertex_ai.init(project=PROJECT_ID, location=LOCATION)
!gcloud config set project {PROJECT_ID}

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Updated property [core/project].


In [22]:
# !gcloud auth application-default login

## Build serving container

In [2]:
REPO_DOCKER_PATH_PREFIX = 'src'
SERVING_SUB_DIR = 'serving'
SERVING_DOCKERNAME = 'triton-cpr'

In [37]:
# Make the training subfolder
! rm -rf {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}
! mkdir {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}
! touch {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/__init__.py

nvtabular==1.3.3 (for prediction only per ronnay AK)

In [38]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/requirements.txt
uvicorn[standard]==0.15.0
gunicorn==20.1.0
fastapi==0.68.1
google-cloud-aiplatform
git+https://github.com/NVIDIA-Merlin/models.git
nvtabular==1.3.3
gcsfs
google-cloud-storage

Writing src/serving/requirements.txt


In [39]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/predictor.py
import nvtabular as nvt
import pandas as pd
import os
import json
import merlin.models.tf as mm
from nvtabular.loader.tf_utils import configure_tensorflow
configure_tensorflow()
import tensorflow as tf


# These are helper functions that ensure the dictionary input is in a certain order and types are preserved
# this is to get scalar values to appear first in the dict to not confuse pandas with lists https://github.com/pandas-dev/pandas/issues/46092
reordered_keys = [
    'collaborative', 
    'album_name_pl', 
    'artist_genres_pl', 
    'artist_name_pl', 
    'artist_pop_can', 
    'description_pl', 
    'duration_ms_songs_pl', 
    'n_songs_pl', 
    'name', 
    'num_albums_pl', 
    'num_artists_pl', 
    'track_name_pl', 
    'track_pop_pl', 
    'duration_ms_seed_pl', 
    'pid', 
    'track_uri_pl'
]

float_num_fix = ['n_songs_pl','num_albums_pl','num_artists_pl','duration_ms_seed_pl']
float_list_fix = ['track_pop_pl', 'duration_ms_songs_pl']
    
def fix_list_num_dtypes(num_list):
    "this fixes lists of ints to list of floats converted in json input"
    return [float(x) for x in num_list]

def fix_num_dtypes(num):
    "this fixes ints and casts to floats"
    return float(num)

def fix_types(k, v):
    if k in float_num_fix:
        return fix_num_dtypes(v)
    if k in float_list_fix:
        return fix_list_num_dtypes(v)
    else:
        return v

def create_pandas_instance(inputs):
    """
    Helper function to reorder the input to have a sclar first for pandas
    And fix the types converted when data is imported by fastAPI
    """
    if type(inputs) == list:
        header = inputs[0]
        reordered_header_dict = {k: fix_types(k,header[k]) for k in reordered_keys}
        pandas_instance = pd.DataFrame.from_dict(reordered_header_dict, orient='index').T
        if len(inputs) > 1:
            for ti in inputs[1:]:
                reordered_dict = {k: fix_types(k,ti[k]) for k in reordered_keys}
                pandas_instance = pandas_instance.append(pd.DataFrame.from_dict(reordered_dict, orient='index').T)
    else:
        reordered_dict = {k: fix_types(k,inputs[k]) for k in reordered_keys}
        pandas_instance = pd.DataFrame.from_dict(reordered_dict, orient='index').T
    return pandas_instance

class Predictor():
    """Interface of the Predictor class for Custom Prediction Routines.
    The Predictor is responsible for the ML logic for processing a prediction request.
    Specifically, the Predictor must define:
    (1) How to load all model artifacts used during prediction into memory.
    (2) The logic that should be executed at predict time.
    When using the default PredictionHandler, the Predictor will be invoked as follows:
      predictor.postprocess(predictor.predict(predictor.preprocess(prediction_input)))
    """
    def __init__(self):
        return
    
    def load(self, artifacts_uri):
        """Loads the model artifact.
        Args:
            artifacts_uri (str):
                Required. The value of the environment variable AIP_STORAGE_URI.
        """
        test_bucket = 'jt-merlin-scaling'
        self.model = tf.keras.models.load_model(os.path.join(artifacts_uri, "query-tower"))
        # self.workflow = nvt.Workflow.load(os.path.join(artifacts_uri, "workflow/2t-spotify-workflow")) # TODO: parameterize
        self.workflow = nvt.Workflow.load(os.path.join(test_bucket, "nvt-last5-v1full/nvt-analyzed"))
        # self.workflow = nvt.Workflow.load('gs://jt-merlin-scaling/nvt-last5-v1full/nvt-analyzed') # TODO: parametrize
        self.workflow = self.workflow.remove_inputs(
            [
                'track_pop_can', 
                'track_uri_can', 
                'duration_ms_can', 
                'track_name_can', 
                'artist_name_can',
                'album_name_can',
                'album_uri_can',
                'artist_followers_can', 
                'artist_genres_can',
                'artist_name_can', 
                'artist_pop_can',
                'artist_pop_pl',
                'artist_uri_can', 
                'artists_followers_pl'
            ]
        )
        self.loader = None # will load this after first load
        
        return self
        
    def preprocess(self, prediction_input):
        """Preprocesses the prediction input before doing the prediction.
        Args:
            prediction_input (Any):
                Required. The prediction input that needs to be preprocessed.
        Returns:
            The preprocessed prediction input.
        """
        # handle different input types, can take a dict or list of dicts
        pandas_instance = create_pandas_instance(prediction_input[0])
        transformed_inputs = nvt.Dataset(pandas_instance)
        transformed_instance = self.workflow.transform(transformed_inputs)
        return transformed_instance

    def predict(self, instances):
        """Performs prediction.
        Args:
            instances (Any):
                Required. The instance(s) used for performing prediction.
        Returns:
            Prediction results.
        """  
        if self.loader is None:
            self.loader = mm.Loader(instances, batch_size=32, shuffle=False)
        else:
            self.loader.data = instances #this is faster we don't want tokeep loading dataloder
        batch = next(iter(self.loader))
        return self.model(batch[0])

Writing src/serving/predictor.py


In [40]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/main.py

from fastapi import FastAPI, Request
import json
import numpy as np
import os
import logging

from google.cloud import storage
from predictor import Predictor

app = FastAPI()

predictor_instance = Predictor()

loaded_predictor = predictor_instance.load(artifacts_uri = os.environ['AIP_STORAGE_URI'])

@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {}

@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()
    instances = body["instances"]
    preprocessed_inputs = loaded_predictor.preprocess(instances)
    outputs = loaded_predictor.predict(preprocessed_inputs)

    return {"predictions": outputs.numpy().tolist()}

Writing src/serving/main.py


In [41]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/prestart.sh
#!/bin/bash
export PORT=$AIP_HTTP_PORT

Writing src/serving/prestart.sh


In [42]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/instances.json
{"instances": {"collaborative": "false", "album_name_pl": ["There's Really A Wolf", "Late Nights: The Album", "American Teen", "Crazy In Love", "Pony"], "album_uri_can": "spotify:album:5l83t3mbVgCrIe1VU9uJZR", "artist_followers_can": 4339757.0, "artist_genres_can": "'hawaiian hip hop', 'rap'", "artist_genres_pl": ["'hawaiian hip hop', 'rap'", "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'", "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'", "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"], "artist_name_can": "Russ", "artist_name_pl": ["Russ", "Jeremih", "Khalid", "Beyonc\u00c3\u00a9", "William Singe"], "artist_pop_can": 82.0, "artist_pop_pl": [82.0, 80.0, 90.0, 87.0, 65.0], "artist_uri_can": "spotify:artist:1z7b1Pr1rSlvWRzsW3HOrS", "artists_followers_pl": [4339757.0, 5611842.0, 15046756.0, 30713126.0, 603837.0], "description_pl": "", "duration_ms_can": 237322.0, "duration_ms_songs_pl": [237506.0, 217200.0, 219080.0, 226400.0, 121739.0], "n_songs_pl": 8.0, "name": "Lit Tunes ", "num_albums_pl": 8.0, "num_artists_pl": 8.0, "track_name_can": "We Just Havent Met Yet", "track_name_pl": ["Losin Control", "Paradise", "Location", "Crazy In Love - Remix", "Pony"], "track_pop_can": 57.0, "track_pop_pl": [79.0, 58.0, 83.0, 71.0, 57.0], "duration_ms_seed_pl": 51023.1, "pid": 1, "track_uri_can": "spotify:track:0VzDv4wiuZsLsNOmfaUy2W", "track_uri_pl": ["spotify:track:4cxMGhkinTocPSVVKWIw0d", "spotify:track:1wNEBPo3nsbGCZRryI832I", "spotify:track:152lZdxL1OR0ZMW6KquMif", "spotify:track:2f4IuijXLxYOeBncS60GUD", "spotify:track:4Lj8paMFwyKTGfILLELVxt"]}}

Writing src/serving/instances.json


In [43]:
%%writefile {REPO_DOCKER_PATH_PREFIX}/Dockerfile.{SERVING_DOCKERNAME}

FROM nvcr.io/nvidia/merlin/merlin-tensorflow:22.09
WORKDIR /serving 

COPY ./serving/requirements.txt /requirements.txt
RUN pip install -r /requirements.txt

COPY ./serving /serving
EXPOSE 80
    
CMD ["sh", "-c", "uvicorn main:app --host 0.0.0.0 --port $AIP_HTTP_PORT"]

Overwriting src/Dockerfile.triton-cpr


## Copy serving app to GCS

**TODO**
> currently getting gsutil auth issues.. refactor this section with `gsutil -cp ...`

In [44]:
# !gcloud auth login

from google.cloud import storage
from google.cloud.storage.bucket import Bucket
from google.cloud.storage.blob import Blob

def _upload_blob_gcs(gcs_uri, source_file_name, destination_blob_name, project):
    """Uploads a file to GCS bucket"""
    client = storage.Client(project=project)
    blob = Blob.from_string(os.path.join(gcs_uri, destination_blob_name))
    blob.bucket._client = client
    blob.upload_from_filename(source_file_name)

In [45]:
LOCAL_FILENAME_docker = f'{REPO_DOCKER_PATH_PREFIX}/Dockerfile.{SERVING_DOCKERNAME}'
LOCAL_FILENAME_main = f'{REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/main.py'
LOCAL_FILENAME_predictor = f'{REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/predictor.py'
LOCAL_FILENAME_prestart = f'{REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/prestart.sh'
LOCAL_FILENAME_reqs = f'{REPO_DOCKER_PATH_PREFIX}/{SERVING_SUB_DIR}/requirements.txt'

GCS_MODEL_DIR_PATH = 'test-2tower-merlin-tf-jtv1/run-v5-20221108-163948/model-dir'
GCS_SERVING_DIR_PATH = f'{GCS_MODEL_DIR_PATH}/serving'

DESTINATION_FILENAME_docker = f'{GCS_SERVING_DIR_PATH}/Dockerfile.{SERVING_DOCKERNAME}'
DESTINATION_FILENAME_main = f'{GCS_SERVING_DIR_PATH}/app/main.py'
DESTINATION_FILENAME_predictor = f'{GCS_SERVING_DIR_PATH}/app/predictor.py'
DESTINATION_FILENAME_prestart = f'{GCS_SERVING_DIR_PATH}/app/prestart.sh'
DESTINATION_FILENAME_reqs = f'{GCS_SERVING_DIR_PATH}/app/requirements.txt'

# print(f"LOCAL_FILENAME_docker : {LOCAL_FILENAME_docker}")
# print(f"LOCAL_FILENAME_main : {LOCAL_FILENAME_main}")
# print(f"LOCAL_FILENAME_predictor : {LOCAL_FILENAME_predictor}")
# print(f"LOCAL_FILENAME_prestart : {LOCAL_FILENAME_prestart}")
# print(f"LOCAL_FILENAME_reqs : {LOCAL_FILENAME_reqs}\n")
print(f"GCS_MODEL_DIR_PATH : {GCS_MODEL_DIR_PATH}\n")
print(f"DESTINATION_FILENAME_docker : {DESTINATION_FILENAME_docker}")

GCS_MODEL_DIR_PATH : test-2tower-merlin-tf-jtv1/run-v5-20221108-163948/model-dir

DESTINATION_FILENAME_docker : test-2tower-merlin-tf-jtv1/run-v5-20221108-163948/model-dir/serving/Dockerfile.triton-cpr


In [46]:
_upload_blob_gcs(BUCKET_URI, LOCAL_FILENAME_docker,DESTINATION_FILENAME_docker, PROJECT_ID)
_upload_blob_gcs(BUCKET_URI, LOCAL_FILENAME_main,DESTINATION_FILENAME_main, PROJECT_ID)
_upload_blob_gcs(BUCKET_URI, LOCAL_FILENAME_predictor,DESTINATION_FILENAME_predictor, PROJECT_ID)
_upload_blob_gcs(BUCKET_URI, LOCAL_FILENAME_prestart,DESTINATION_FILENAME_prestart, PROJECT_ID)
_upload_blob_gcs(BUCKET_URI, LOCAL_FILENAME_reqs,DESTINATION_FILENAME_reqs, PROJECT_ID)

In [24]:
# !gsutil ls gs://jt-merlin-scaling/test-2tower-merlin-tf-jtv1/run-v5-20221108-163948/model-dir/serving

### Build Serving Image

In [47]:
# Docker definitions for training
SERVING_VERSION = 'v2'
IMAGE_NAME = f'merlin-triton-serving-{SERVING_VERSION}'
IMAGE_URI = f'gcr.io/{PROJECT_ID}/{IMAGE_NAME}'

DOCKERNAME = f'{SERVING_DOCKERNAME}'
MACHINE_TYPE ='e2-highcpu-32'
FILE_LOCATION = './src'

In [48]:
# FILE_LOCATION = './src'
# ! gcloud builds submit --config src/cloudbuild.yaml --substitutions _DOCKERNAME=$DOCKERNAME,_IMAGE_URI=$IMAGE_URI,_FILE_LOCATION=$FILE_LOCATION --timeout=2h --machine-type=e2-highcpu-8

In [49]:
! gcloud builds submit --config src/cloudbuild.yaml \
    --substitutions _DOCKERNAME=$DOCKERNAME,_IMAGE_URI=$IMAGE_URI,_FILE_LOCATION=$FILE_LOCATION \
    --timeout=2h \
    --machine-type=$MACHINE_TYPE

Creating temporary tarball archive of 32 file(s) totalling 1.2 MiB before compression.
Uploading tarball of [.] to [gs://hybrid-vertex_cloudbuild/source/1667942521.471558-c00443817c854a57bea99bb588d7350a.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/hybrid-vertex/locations/global/builds/1cbd8efa-7ce7-488d-9fed-d58ebfd936ab].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/1cbd8efa-7ce7-488d-9fed-d58ebfd936ab?project=934903580331 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "1cbd8efa-7ce7-488d-9fed-d58ebfd936ab"

FETCHSOURCE
Fetching storage object: gs://hybrid-vertex_cloudbuild/source/1667942521.471558-c00443817c854a57bea99bb588d7350a.tgz#1667942521926964
Copying gs://hybrid-vertex_cloudbuild/source/1667942521.471558-c00443817c854a57bea99bb588d7350a.tgz#1667942521926964...
/ [1 files][229.8 KiB/229.8 KiB]                                                
Operation completed over 1 objects/229.

# Deploy Query Model with Vertex AI

## Upload to Model Regsitry

In [51]:
MODEL_DISPLAY_NAME = "mm_qtower_test_v2"
MODEL_ARTIFACT_URI = 'gs://jt-merlin-scaling/test-2tower-merlin-tf-jtv1/run-v5-20221108-163948/model-dir'

In [52]:
model = vertex_ai.Model.upload(
        display_name=MODEL_DISPLAY_NAME,
        artifact_uri=MODEL_ARTIFACT_URI,
        serving_container_image_uri=IMAGE_URI,
        serving_container_predict_route='/predict',
        serving_container_health_route='/health',
        serving_container_command=["sh", "-c", "uvicorn main:app --host 0.0.0.0 --port $AIP_HTTP_PORT"],
        serving_container_args='--gpus all',
        sync=True,
    )

Creating Model
Create Model backing LRO: projects/934903580331/locations/us-central1/models/2473175484821667840/operations/4582510820981735424
Model created. Resource name: projects/934903580331/locations/us-central1/models/2473175484821667840@1
To use this Model in another session:
model = aiplatform.Model('projects/934903580331/locations/us-central1/models/2473175484821667840@1')


## Deploy to Vertex AI Endpoint

In [ ]:
endpoint = model.deploy(
    machine_type="n1-standard-4",
    accelerator_type="NVIDIA_TESLA_T4",
    accelerator_count=1
)

Creating Endpoint
Create Endpoint backing LRO: projects/934903580331/locations/us-central1/endpoints/6782918018075721728/operations/15649692595519488
Endpoint created. Resource name: projects/934903580331/locations/us-central1/endpoints/6782918018075721728
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/934903580331/locations/us-central1/endpoints/6782918018075721728')
Deploying model to Endpoint : projects/934903580331/locations/us-central1/endpoints/6782918018075721728
Deploy Endpoint model backing LRO: projects/934903580331/locations/us-central1/endpoints/6782918018075721728/operations/8968242801854644224


## Test Deployed Endpoint

In [ ]:
## Ground truth candidate:
    # 'album_uri_can': 'spotify:album:5l83t3mbVgCrIe1VU9uJZR', 
    # 'artist_name_can': 'Russ', 
    # 'track_name_can': 'We Just Havent Met Yet', 
## TODO - we have to overload with candidate data because of the workflow transform, add overloaded values in the predictor
TEST_INSTANCE = {'collaborative': 'false',
                 'album_name_pl': ["There's Really A Wolf", 'Late Nights: The Album',
                       'American Teen', 'Crazy In Love', 'Pony'], 
                 'artist_genres_pl': ["'hawaiian hip hop', 'rap'",
                       "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'",
                       "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'",
                       "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"], 
                 'artist_name_pl': ['Russ', 'Jeremih', 'Khalid', 'Beyonc\xc3\xa9',
                       'William Singe'], 
                 'artist_pop_can': 82.0, 
                 'description_pl': '', 
                 'duration_ms_songs_pl': [237506.0, 217200.0, 219080.0, 226400.0, 121739.0], 
                 'n_songs_pl': 8.0, 
                 'name': 'Lit Tunes ', 
                 'num_albums_pl': 8.0, 
                 'num_artists_pl': 8.0, 
                 'track_name_pl': ['Losin Control', 'Paradise', 'Location',
                       'Crazy In Love - Remix', 'Pony'], 
                 'track_pop_pl': [79.0, 58.0, 83.0, 71.0, 57.0],
                 'duration_ms_seed_pl': 51023.1,
                 'pid': 1,
                 'track_uri_pl': ['spotify:track:4cxMGhkinTocPSVVKWIw0d',
                       'spotify:track:1wNEBPo3nsbGCZRryI832I',
                       'spotify:track:152lZdxL1OR0ZMW6KquMif',
                       'spotify:track:2f4IuijXLxYOeBncS60GUD',
                       'spotify:track:4Lj8paMFwyKTGfILLELVxt']
                     }

In [ ]:
#make a prediction

endpoint.predict(instances=[[TEST_INSTANCE, TEST_INSTANCE]])